$$
%\newcommand{\oneform}[1]{{\vphantom{{#1}}}^{1}\!{#1}{\vphantom{#1}}}
% \newcommand{\oneform}[1]{\overset{1}{#1}{\vphantom{#1}}}
%\newcommand{\volform}[1]{{\vphantom{{\omega}}}^{#1}\!{\omega}{\vphantom{\omega}}}
% \newcommand{\volform}[1]{\overset{#1}{\omega}{\vphantom{\omega}}}
%\renewcommand{\vector}[1]{\boldsymbol{#1}}
% \newcommand{\curve}[1]{{#1}}
% \newcommand{\fbasis}[1]{{d#1}}
% \newcommand{\uprm}[1]{^{\mathrm{#1}}}
% \newcommand{\tensor}[1]{\mathbf{#1}}
% \newcommand{\norm}[1]{||#1||}
$$

# Overview

The problem of interest is the coupled response of a soft spherical body that is exposed to electric
and magnetic fields. The body is assumed to consists of an almost incompressible, elastic, linearly dielectric and 
(para)magnetic material.
This problem has a rather simple setup but is representative enough for comparisons of computational performance.

As a starting point, a FE implementation of the electro-mechanical problem is provided.
The tasks of the project are given below the implementation.

In [ ]:
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
import time
import numpy as np
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.nonlinearsolvers import Newton, NewtonSolver
import pathlib

# Geometry

In [ ]:
def createGeometryAndMesh(rmaxh,Rmaxh, drawSpace, drawMesh):

    global mesh, geo, AIR, BODY

    octant = Box((0,0,0), (2*R, 2*R, 2*R))
    everywhere = Sphere((0, 0, 0), R) * octant
    body = Sphere((0, 0, 0), r) * octant
    air = everywhere - body
    body.mat("body")
    air.mat("air")
    all_space = Glue([body, air])

    for f in all_space.faces:
        f.bc("outer")
    
    for f in body.faces:
        f.bc("inner")

    for f in all_space.faces[X < 1e-3]:
        f.bc("YZ_symm")
        
    for f in all_space.faces[Y < 1e-3]:
        f.bc("ZX_symm")
        
    for f in all_space.faces[Z < 1e-3]:
        f.bc("XY_symm")

    body.maxh =  rmaxh
    air.maxh = Rmaxh
    geo = OCCGeometry(all_space, dim=3)
    ngmesh = geo.GenerateMesh()
    mesh = Mesh(ngmesh)
    #mesh.Refine()
    mesh.Curve(3)
    
    AIR = mesh.Materials("air")
    BODY = mesh.Materials("body")


    if drawSpace == True:
        DrawGeo(all_space)

    if drawMesh == True:
        Draw(mesh)  
    

# Function spaces and grid functions

In [ ]:
def createSpacesAndGfs(p_order):

    global fes, fes_u, fes_phi, fes_E, fes_s

    global gfu, gfphi, gfF, gfE, gfe, gfe_ext, gfd, gfD, gfPK1, gfsigma, gfsol

    global u, phi

    fes_u = VectorH1(mesh, order=p_order, 
                    dirichletx="outer|YZ_symm", 
                    dirichlety="outer|ZX_symm",
                    dirichletz="outer|XY_symm")

    fes_phi = H1(mesh, order=p_order, dirichlet="XY_symm")

    fes = fes_u * fes_phi

    u, phi = fes.TrialFunction()

    gfsol = GridFunction(fes)
    gfu, gfphi = gfsol.components

    # E is a function of the derivatives of phi but not phi directly. 
    # Thus, for it's a polynomial degree one may choose that of phi minus 1.
    fes_E = VectorL2(mesh, order=p_order-1)

    # The gridfunction storing the actual value of the Eulerian electric field - e eulerian, E Lagrangian
    gfe = GridFunction(fes_E)

    # The external part
    gfe_ext = GridFunction(fes_E)

    # Stores the Lagrangian electric field
    gfE = GridFunction(fes_E)

        # Eulrian
    gfd = GridFunction(fes_E)

    # Lagrangian
    gfD = GridFunction(fes_E)

    fes_s = MatrixValued(L2(mesh, order=p_order-1))

    # We want the "Cauchy-type" total stress -> "sigma"
    gfsigma = GridFunction(fes_s)

    # The surface force density per referential area
    gfPK1 = GridFunction(fes_s) # first piola kirchhof Tensor! not symmetric
    
    #reuse fes_s for the tangent map
    gfF = GridFunction(fes_s)

    

In [ ]:
# GeometryAndMesh(rmaxh,Rmaxh, drawSpace, drawMesh):

R = 10
r = 1

createGeometryAndMesh(r/3,R/3, False, False)
createSpacesAndGfs(3)

### Collect grid functions

In [ ]:
pp_gf_dict = {
    "u": gfu,
    "phi": gfphi,
    "F": gfF,
    "E": gfE,
    "e": gfe,
    "e_ext": gfe_ext,
    "D": gfD,
    "d": gfd,
    "PK1": gfPK1,
    "sigma": gfsigma,
}

# Kinematics

In [ ]:
I = Id(mesh.dim) # Retruns the 3x3 identity matrix

def F(u):
    return I + Grad(u)


def Cof(F):
    return Det(F) * Inv(F)


def InvCof(F):
    return 1/Det(F) * F


# The right Cauchy Green tensor (metric can be omitted for brevity) this the big fat dot???????? 
g = I
def C(F):
    return F.trans * g * F

# The external electric field CF is a coefficient function expression, Dear matthias you are a good coder! (0, 0, 0)
e_ext = CF(tuple(Parameter(0) for ii in range(mesh.dim)))

# The Lagrangian electric (self) field
def E(phi):
    return -Grad(phi)

## Neo-Hookean material

In [ ]:
# E_... -> Young's modulus
E_air, nu_air = Parameter(0.001), Parameter(0.2)
E_body, nu_body = Parameter(0.1), Parameter(0.499)

# shorthands
I_C = Trace
III_C = Det

def Psi_NH(C, E, nu):
    mu  = E / 2 / (1+nu) # shear modulus
    lam = E * nu / ((1+nu)*(1-2*nu))
    
    # NOTE: we use I_C(C), III_C(C)...
    return mu/2 * (I_C(C) - 3 - log(III_C(C))) + lam/8 * (log(III_C(C)))**2

## Electrostatic energy density

In [ ]:
epsilon_0 = 8.854*1e-6 # permittivity of vacuum in units corresponding to [E] = MV/m and [D] = C/m^2
epsilon_r_body = Parameter(5) # 5 times as "permitting" as vacuum 
epsilon_r_air = Parameter(1) # air treated as vacuum in terms of permittivity
# The right Cauchy Green Tensor "describes" the shape, therefore involved in the ES functional.
def Psi_ES(C, E, epsilon_r):
    J = sqrt(III_C(C)) # Be careful if one uses C or F as arg of III_C
    return -1/2 * epsilon_0 * epsilon_r * InnerProduct(Inv(C) * E, E) * J
# InnerProduct in NGSolve here is row column multiplication -> classical dot product
# if no deformation involved, no C involved in Psi_ES, connected to the Lie-Rates -> coming lectures.

## Combined

In [ ]:
def generate_Psi_dict(C, E):
    return {AIR: Psi_ES(C, E, epsilon_r_air) + Psi_NH(C, E_air, nu_air), 
            BODY: Psi_ES(C, E, epsilon_r_body) + Psi_NH(C, E_body, nu_body),}

Put things in a dict for having them accessible by domain name.

## Postprocessing definitions

In [ ]:
def generate_pp_dict(F, E, Psi_dict=None): # capital pi
    F.MakeVariable() # Needed to take derivatives of expressions with respects to it!
    E.MakeVariable() 
    J = Det(F) 
    Psi_dict = generate_Psi_dict(C(F), E) if Psi_dict is None else Psi_dict
    pp_dict = {
        "E": E,
        "e": Inv(F.trans) * E, 
        "e_ext": e_ext,
        "F": F,
        "D": {domain: -Psi.Diff(E) for domain, Psi in Psi_dict.items()},
        "d": {domain: -InvCof(F) * Psi.Diff(E) for domain, Psi in Psi_dict.items()}, 
        "PK1": {domain: Psi.Diff(F) for domain, Psi in Psi_dict.items()},
        "sigma": {domain: Psi.Diff(F)*InvCof(F).trans for domain, Psi in Psi_dict.items()},
    }
    return {kk: ({k2: v2.Compile() for k2, v2 in vv.items()} if isinstance(vv, dict) else vv.Compile()) # k = keys, v = values
            for kk, vv in pp_dict.items()} # its good code yes..


In [ ]:
# Create postprocessing dictionary
pp_dict = generate_pp_dict(F(gfu), E(gfphi))

In [ ]:
def pp(pp_gf_dict, pp_dict, vtk, time=None): # pp = postprocessing
    # interpolate - yes only interpolate
    for key, value in pp_gf_dict.items():
        if key in pp_dict:
            if isinstance(pp_dict[key], dict):
                for domain, expr in pp_dict[key].items():
                    value.Interpolate(expr, definedon=domain)
            else:
                value.Interpolate(pp_dict[key])
    vtk.Do(time=time)

# Govering potential - FInally some NGSOlve :-)

In [ ]:
Pi = BilinearForm(fes, symmetric=True)
_F = F(u) # u is trialfunction
Pi += Variation(
    sum([Psi.Compile() * dx(domain) for domain, Psi in generate_Psi_dict(C(F(u)), E(phi)).items()])
    #All the Energydensity Integrals over all domains WITHIN domain.
).Compile() 

N = specialcf.normal(mesh.dim) 
Pi += Variation(phi * epsilon_0 * e_ext * N * ds(mesh.Boundaries("outer"))).Compile() # we add the external electric field to the variation

In [ ]:
# Create the vector holding the discrete variation
rhs = gfsol.vec.CreateVector()

e_ext[2].Set(0)
gfsol.vec[:] = 0 # Could very well be that we dont need this. 

# Compute the variation; evaluate with the data of gfu
Pi.Apply(gfsol.vec, rhs) #.Applies Variational Formulation to an input vector and returns the result

#help(BilinearForm)
print(gfsol.space.ndof)

In [ ]:
Norm(rhs)

## A modified version that does not suffer from spurious deformation

In [ ]:
def assemblePiMod():

    global Pi_mod, Pi_interface, interface_dofs

   # interface dofs:
    mech_interface_dofs = np.array(fes.GetDofs(mesh.Boundaries("inner")))
    mag_dof_range = fes.Range(1)
    mech_interface_dofs[np.arange(mag_dof_range.start, mag_dof_range.stop, mag_dof_range.step)] = False
    interface_dofs = np.where(mech_interface_dofs)[0]

    # The governing potential. This time, we do not assume symmetry because we'll apply a modification to
    # the resulting matrix that renders the system non-symmetric.
    Pi_mod = BilinearForm(fes, symmetric=False)
    Pi_mod += Variation(
        sum([Psi.Compile() * dx(domain) for domain, Psi in generate_Psi_dict(C(F(u)), E(phi)).items()])
    )
    N = specialcf.normal(mesh.dim)
    Pi_mod += Variation(phi * epsilon_0 * e_ext * N * ds(mesh.Boundaries("outer")))

    # The (neg.) *mechanical* contribution of the air at the interface. Note the "CF((0,0,0))"
    # in the expression below, which effectly forces the electrostatic contribution to zero.
    Pi_interface = BilinearForm(fes, symmetric=False)
    Pi_interface += Variation(
        sum([(-Psi).Compile() * dx(domain) for domain, Psi in generate_Psi_dict(C(_F), CF((0,0,0))).items() 
            if domain == AIR])
    )

class BFWrapper: # Bilinearform Wrapper which uses static condensation / schauder complement to get inverse
    def __init__(self, a, a_interface, interface_dofs, gfsol):
        self._a = a
        self._a_interface = a_interface
        
        if self._a.condense or self._a_interface.condense:
            raise ValueError("Static condensation not supported.")
        
        self._interface_dofs = interface_dofs
        self._u = gfsol
        self._interface_indices = None
        self._r_interface = gfsol.vec.CreateVector()
    
    @property
    def mat(self):
        return self._a.mat
    
    @property
    def condense(self):
        return self._a.condense
    
    def _setup_interface_data(self, force=False):
        if self._interface_indices is None or force:
            try:
                mrows, mcols, ivals = self._a_interface.mat.COO()
                mrows_np = mrows.NumPy()
                print("\nexpensive, non-optimal operation (but only once)...")
                self._interface_indices = np.hstack([np.where(mrows_np == d) for d in self._interface_dofs]).flatten()
                print("...done\n")
                self._mat_as_vec = self._a.mat.AsVector().FV().NumPy()
                self._mat_i_as_vec = self._a_interface.mat.AsVector().FV().NumPy()
            except TypeError as e:
                self._a.AssembleLinearization(self._u.vec)
                self._a_interface.AssembleLinearization(self._u.vec)
                self._setup_interface_data()
    
    def Apply(self, vec, rhs):
        self._setup_interface_data()
        self._a.Apply(vec, rhs)
        self._a_interface.Apply(vec, self._r_interface)
        rhs.FV().NumPy()[self._interface_dofs] += self._r_interface.FV().NumPy()[self._interface_dofs]
    
    def AssembleLinearization(self, vec):
        self._a.AssembleLinearization(vec)
        self._a_interface.AssembleLinearization(vec)
        self._setup_interface_data()
        self._mat_as_vec[self._interface_indices] += self._mat_i_as_vec[self._interface_indices]

In [ ]:
assemblePiMod()
Pi2 = BFWrapper(Pi_mod, Pi_interface, interface_dofs, gfsol)

# Run the problem

In [ ]:
def resetProblem():

    global _Pi, gfsol_ba

    e_ext[2].Set(0)
    gfsol.vec[:] = 0
    gfsol_ba = GridFunction(gfsol.space) # ba?
    _Pi = Pi2
    E_body.Set(0.05)
    E_air.Set(E_body.Get()/100)  # why set the Young´s Moduli equal?



In [ ]:

# odir = pathlib.Path("output2")

# otherwise
# _Pi = Pi
# E_body.Set(0.05)
# E_air.Set(E_body.Get() / 100) # --> will fail due to excessive spurious deformation in air domain
# odir = pathlib.Path("output")

# odir.mkdir(exist_ok=True)
# vtk = VTKOutput(
#     mesh,
#     coefs=list(pp_gf_dict.values()),
#     names=list(pp_gf_dict.keys()),
#     subdivision=2,
#     filename=str(odir / "output")
# )


In [ ]:
def run_load_step(dz):
    #with TaskManager(pajetrace=10**8):
    with TaskManager(pajetrace=10**8):
    
        e_ext[2].Set(dz)
        print(f"\ntrying to solve for load parameter val = {str(dz)}...\n")
        success, niter = Newton(_Pi, gfsol, inverse="pardiso", maxit=15)
        
        # success, niter = Newton(_Pi, gfsol, inverse="sparsecholesky", maxit=20)
        #success, niter = Newton(_Pi, gfsol,inverse = "umfpack", maxit=15)
        if success != 0:
            raise Exception("Newton did not converge")
            
        # c = Preconditioner(Pi, "local") # 'Register' c to a BEFORE assembly
        # Pi.AssembleLinearization(gfsol.vec)
        # inv = CGSolver(Pi.mat, c.mat, maxsteps=1000) # CGMethod not possible, GMRES with preconditioner


        # pp(pp_gf_dict, pp_dict, vtk, time=dz)
        # gfsol_ba.vec.data = gfsol.vec # ba -> backup, happens sometimes that doesnt converge, wanna have the last state to recover.

        # print(f"\nsuccessfully solved for load parameter dz = {str(dz)}")
        # print(f"z-displacement at (0,0,r) = {gfu(mesh(0,0,r))[2]:e}")
        # print("\n{:s}\n".format("-" * 80))

In [ ]:
# SetNumThreads(12)
# start = time.time()
# for val in np.linspace(0, 10, 4):
#     run_load_step(val)
# elapsed_time = time.time() - start

# print(elapsed_time)

#loadstep(0,30,5) and  maxh = r/4 -> no convergence
#loadstep(0,30m,7) maxh = r/6 -> elapsed time = 81 seconds..
#loadstep(0,10,4) maxh = r/3 -> elapsed time = 9.227369785308838
#loadstep(0,10,4) maxh = r/4 -> elapsed time = 18.688396453857422
#loadstep(0,10,4) maxh = r/5 -> elapsed time = 23.64073395729065
#loadstep(0,10,4) maxh = r/6 -> elapsed time = 42.09492039680481
#loadstep(0,10,4) maxh = r/7 -> elapsed time = 80.42692923545837
#loadstep(0,10,4) maxh = r/7 -> elapsed time = 155.19630765914917





Problem Convergence Characteristics
12 Threads @ 4.3 GHz
Newton(Pi, gfsol, inverse="sparsecholesky", maxit=10)

In [ ]:
print(gfsol.space.ndof)
# GeometryAndMesh(rmaxh,Rmaxh, drawSpace, drawMesh):

createGeometryAndMesh(r/6,R/6, False, False)
createSpacesAndGfs(3)
resetProblem()
assemblePiMod()
Pi2 = BFWrapper(Pi_mod, Pi_interface, interface_dofs, gfsol)

print(gfsol.space.ndof)
SetNumThreads(12)
start = time.time()
for val in np.linspace(0, 1, 5):
    run_load_step(val)
elapsed_time = time.time() - start
print(elapsed_time)
# print(mesh.GetMaterials())
# print(mesh.GetBoundaries())


print(gfsol.space.ndof)


# Tasks

NOTE: Choose the tasks that you find most interesting and try to spend something around 3 hours in total on implementation, computations and processing of results. Do not hesitate to ask for assistance when you got stuck.
Unfinished tasks might be dealt with in forthcoming exercises.


## Linear solvers for the coupled problem

The default setting in the notebook is "pardiso", a direct solver. 
Try some iterative solvers (system is indefinite and non-symmetric) with preconditioning and 
play around with mesh resolution and polynomial degree.
How do the methods perform in terms of computation time? Can you say some about the scaling of computation
time wrt. to mesh resolution and polynomial degree.

- Pardiso leads to bad results when in increasing the polynomial    
degree and mesh resolution (Pardiso is hopefully non-direct Solver.)

- Sparse-Cholesky does not converge for default Newton

- Umfpack and Mumps not yet possible, need to compile NGSolve with it?

## After Exercise Discussion

- get the ndofs for all the sizes and redo the plots for the timings.

- staggered scheme, use pardiso, dont use BFWrapper, if it works, maybe use iterative solver with precond

- investigate the time-complexity payoff of staggered scheme and monolithic scheme (thats the old one)






## "Staggered" solution scheme

Separate the coupled problem in a purely mechanical and a purely electrostatic BVP. 
The subproblems can be obtained by something like:
```
Pi_mech = BilinearForm(fes_u, symmetric=True)
Pi_mech += Variation(
    sum([Psi.Compile() * dx(domain) for domain, Psi in generate_Psi_dict(C(F(u)), E(gfphi)).items()])
)

Pi_elec = BilinearForm(fes_phi, symmetric=True)
Pi_elec += Variation(
    sum([Psi.Compile() * dx(domain) for domain, Psi in generate_Psi_dict(C(F(gfu)), E(phi)).items()])
)
N = specialcf.normal(mesh.dim)
Pi_elec += Variation(phi * epsilon_0 * e_ext * N * ds(mesh.Boundaries("outer"))).Compile()
```
In this case, one also does not have to fear spurious deformation in air. In turn, the Youngs modulus `E_air` must be set to a very small value, e.g. `E_air.Set(E_body.Get() / 1000)` or even less.

Then, these (in general) *nonlinear* "subproblems" are solved in an alternating manner until convergence (in coupled sense; sol solving one *nonlinear* problem does not perturb the other anymore).
How does this perform in comparison with solving the coupled problem in a "monolithic" way.
Data on scaling is important as well.


Hint 1: It might be that you need to reduce load increments to achieve convergence!

Hint 2: The (linearized) subproblems are much easier to solve! Maybe you find faster iterative and direct solvers.


## Reformulation of the problem

Reformulation the electrostatic part of the problem to the constrained minimization form (with $D$ and $\phi$).
Compare the "direct" implementation wia the corresponding Lagrangian (in optimization sense) function
and an Augmented Lagrangian scheme. What does that mean for the choice of linear solvers in the Newton scheme?